### Testing

In [38]:
from scipy import sparse
import gc
import pandas as pd
from skmultilearn.adapt import MLkNN

In [18]:
f = open('Mediamill_data.txt', 'r', encoding='utf-8')

size = f.readline()
nrows, nfeature,nlabel = [int(s) for s in size.split()]
x_m = [[] for i in range(nrows)]
pos = [[] for i in range(nrows)]
y_m = [[] for i in range(nrows)]

for i in range(nrows):
    line = f.readline()
    temp=[s for s in line.split(sep=' ')]
    pos[i]=[int(s.split(':')[0]) for s in temp[1:]]
    x_m[i]=[float(s.split(':')[1]) for s in temp[1:]]
    for s in temp[0].split(','):
        try:
            int(s)
            y_m[i]=[ int(s) for s in temp[0].split(',')]
        except:
            y_m[i]=[]

In [19]:
x_m[0][0:10]

[0.38088,
 0.49408,
 0.54001,
 0.42293,
 0.15832,
 0.32698,
 0.39086,
 0.52712,
 0.25405,
 0.22373]

In [20]:
pos[0][0:10]

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [21]:
y_m[0]

[11, 31, 65, 67]

converts each row of data into 3 lists: 1. list of x values, 2. list of position corresponding to each x value (feature), list of labels for each feature. We've done something similar to this already using Will's parsing notebook. only difference in we have to cast all the string in the dictionary to int/floats. May want the training labels as a list but that is relatively simple based on Wills code (Thanks Will)

#### Converting Data into Sparse Matrices

**Create an Index for splitting**

Here they are using a separate index files for splitting. I think its realively easy to do the same by using creating a list of numbers in range 0 to `number_of_instance` - 1.

In [22]:
f = open('Mediamill_trSplit.txt',
'r',encoding='utf-8')
train=f.readlines()

f = open('Mediamill_tstSplit.txt',
'r',encoding='utf-8')
test=f.readlines()

In [23]:
train[0:10]

['8480 10458 38036 2847 13816 37526 16917 33089 25502 36772\n',
 '30045 40468 10459 39125 11830 20005 35690 27942 3878 16573\n',
 '36010 37176 5783 12756 8969 41856 23451 15411 39127 17023\n',
 '40540 21431 7341 18650 42196 41813 39768 26397 32625 37533\n',
 '18240 12221 43642 38740 16149 32701 12830 31667 11850 19773\n',
 '26470 4825 40783 13372 8837 21666 33327 40537 37925 1026\n',
 '8072 9892 6876 28835 43537 25751 37953 13597 6179 38515\n',
 '35191 34015 11092 31240 24976 2950 883 34908 15546 10028\n',
 '28304 43731 31912 28297 901 38622 6404 21518 41851 34623\n',
 '33162 9765 31101 18452 5789 22207 42442 11904 8852 36157\n']

We could either make a list for train and val or we could split it into two dataframes using SKlearn

In [24]:
select=0
train_=[int(s.split()[select])-1 for s in train]
test_=[int(s.split()[select])-1 for s in test]

xm_train=[x_m[i] for i in train_]
ym_train=[y_m[i] for i in train_]

xm_test=[x_m[i] for i in test_]
ym_test=[y_m[i] for i in test_]

In [25]:
train_[0:10]

[8479, 30044, 36009, 40539, 18239, 26469, 8071, 35190, 28303, 33161]

In [26]:
test_[0:10]

[35266, 11516, 41468, 218, 28846, 25592, 23298, 18310, 34545, 35581]

In [29]:
xm_train[0][0:10]

[0.42762,
 0.50229,
 0.50827,
 0.45734,
 0.19254,
 0.37528,
 0.38288,
 0.55072,
 0.29573,
 0.25624]

In [30]:
ym_train[0][0:10]

[33, 51, 67, 82, 84]

**Create and empty sparse matrix and fill it with the respective values**

In [31]:

x_train=sparse.lil_matrix((len(train_),nfeature))
for i in range(len(train_)):
    for j in range(len(pos[i])):
        x_train[i,pos[i][j]]=xm_train[i][j]

x_test=sparse.lil_matrix((len(test_),nfeature))
for i in range(len(test_)):
    for j in range(len(pos[i])):
        x_test[i,pos[i][j]]=xm_test[i][j]

# del x_m, xm_train, pos,xm_test
# gc.collect()

y_train=sparse.lil_matrix((len(train_),nlabel))
for i in range(len(train_)):
    for j in ym_train[i]:
        y_train[i,j]=1

y_test=sparse.lil_matrix((len(test_),nlabel))
for i in range(len(test_)):
    for j in ym_test[i]:
        y_test[i,j]=1

# del y_m, ym_train, ym_test
# gc.collect() 

In [32]:
x_train

<30993x120 sparse matrix of type '<class 'numpy.float64'>'
	with 3719160 stored elements in List of Lists format>

In [33]:
pd.DataFrame.sparse.from_spmatrix(x_train)

,0,1,2,3,4,5,6,7,8,9,...,110,111,112,113,114,115,116,117,118,119
0,0.42762,0.50229,0.50827,0.45734,0.19254,0.37528,0.38288,0.55072,0.29573,0.25624,...,0.61509,0.52725,0.62843,0.51795,0.46350,0.16154,0.50734,0.28887,0.54722,0.37215
1,0.53779,0.53994,0.44456,0.47773,0.26428,0.43888,0.38379,0.50511,0.39214,0.34230,...,0.59844,0.49354,0.55102,0.59566,0.52531,0.22885,0.59210,0.39052,0.60540,0.45735
2,0.48325,0.53565,0.52763,0.41462,0.21600,0.39078,0.37018,0.55073,0.33838,0.30226,...,0.61433,0.54472,0.58494,0.55309,0.53552,0.28927,0.57962,0.39791,0.61112,0.46949
3,0.46943,0.50796,0.51921,0.41958,0.20307,0.37521,0.36741,0.52525,0.31567,0.28594,...,0.61385,0.52455,0.57645,0.53418,0.52162,0.23951,0.56270,0.38003,0.59712,0.43565
4,0.49547,0.39359,0.30909,0.37756,0.38400,0.45025,0.29530,0.37839,0.44404,0.39436,...,0.59570,0.43875,0.54259,0.60719,0.59178,0.38929,0.66344,0.54909,0.62459,0.59296
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30988,0.47756,0.51826,0.46713,0.46566,0.24758,0.43069,0.38232,0.53092,0.35431,0.30334,...,0.63315,0.52778,0.58890,0.54333,0.50752,0.22444,0.58225,0.38122,0.60250,0.43735
30989,0.50680,0.54612,0.50393,0.45942,0.22410,0.41076,0.39222,0.54352,0.34856,0.30996,...,0.62522,0.52198,0.59842,0.56623,0.54606,0.32494,0.57975,0.44959,0.62304,0.51442
30990,0.49574,0.48461,0.49412,0.32844,0.22753,0.35207,0.31140,0.46136,0.35642,0.33115,...,0.60251,0.50638,0.58062,0.56627,0.55758,0.26554,0.59777,0.40977,0.60761,0.47042
30991,0.51936,0.46907,0.41931,0.36091,0.26577,0.39528,0.31677,0.45169,0.38111,0.36287,...,0.61732,0.48802,0.57581,0.56308,0.56628,0.41547,0.61252,0.50323,0.61300,0.53528


In [34]:
pd.DataFrame.sparse.from_spmatrix(x_test)

,0,1,2,3,4,5,6,7,8,9,...,110,111,112,113,114,115,116,117,118,119
0,0.36562,0.40611,0.42887,0.32335,0.23190,0.31390,0.29818,0.40125,0.30511,0.29354,...,0.62868,0.51576,0.63152,0.54677,0.54391,0.26247,0.56372,0.41708,0.59142,0.46220
1,0.48805,0.39937,0.36407,0.31907,0.32299,0.38700,0.27720,0.39717,0.40336,0.38580,...,0.59720,0.48567,0.56624,0.56655,0.54680,0.31761,0.63363,0.43679,0.62483,0.50660
2,0.52812,0.45835,0.38483,0.39414,0.31074,0.40204,0.31991,0.42467,0.41762,0.36830,...,0.56364,0.44938,0.50535,0.57870,0.53121,0.26861,0.61298,0.42312,0.59369,0.48862
3,0.51029,0.46597,0.42750,0.33886,0.26418,0.36603,0.30772,0.43657,0.38129,0.35864,...,0.59504,0.51883,0.56104,0.60613,0.56512,0.31981,0.63977,0.49419,0.62352,0.54060
4,0.52804,0.43230,0.35837,0.38457,0.34751,0.44255,0.31717,0.42294,0.43398,0.40195,...,0.59405,0.48729,0.50102,0.60045,0.58862,0.30397,0.64830,0.50305,0.64907,0.54553
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12909,0.51565,0.54271,0.47011,0.47173,0.23594,0.41132,0.38809,0.49471,0.36343,0.31688,...,0.59290,0.53207,0.55456,0.56603,0.54739,0.29020,0.57017,0.40588,0.60114,0.47287
12910,0.36712,0.49058,0.53397,0.41316,0.15350,0.32189,0.37764,0.53593,0.24227,0.20918,...,0.60889,0.51340,0.64502,0.47542,0.44856,0.17147,0.46993,0.29702,0.52178,0.35700
12911,0.35548,0.46606,0.57121,0.34493,0.14163,0.28388,0.32476,0.51012,0.23587,0.21327,...,0.59744,0.53064,0.64358,0.50310,0.51485,0.24012,0.52941,0.35351,0.55164,0.42042
12912,0.47195,0.47846,0.49696,0.33235,0.22476,0.34888,0.30931,0.47373,0.34606,0.31915,...,0.63122,0.50680,0.62493,0.57325,0.54239,0.27853,0.62176,0.42575,0.61799,0.48246


In [35]:
pd.DataFrame.sparse.from_spmatrix(y_train)

,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,100
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30988,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
30989,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
30990,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
30991,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [36]:
pd.DataFrame.sparse.from_spmatrix(y_test)

,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,100
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12909,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12910,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12911,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12912,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [43]:
classifier = MLkNN()

In [44]:
# train
classifier.fit(x_train, y_train)

MLkNN(ignore_first_neighbours=0, k=10, s=1.0)

In [45]:
# predict
predictions = classifier.predict(x_test)

In [47]:
classifier.score(x_test, y_test)

0.14186154560941613